# Importing

In [ ]:
# DataFrame
import pandas as pd

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB


#tensorflow
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Utility
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import string
import pickle

In [ ]:
data_set = pd.read_csv('Project data.csv',encoding='latin', names = ['polarity','id','date','query','user','text'])

# Data visualization

In [ ]:
data_set.shape

(1600000, 6)

In [ ]:
data_set['polarity'].unique()

array([0, 4], dtype=int64)

In [ ]:
# default polarity for +ve is 4 , change it to 1

data_set['polarity'] = data_set['polarity'].replace(4,1)
data_set.head(10)

,polarity,id,date,query,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
5,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
6,0,1467811592,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,mybirch,Need a hug
7,0,1467811594,Mon Apr 06 22:20:03 PDT 2009,NO_QUERY,coZZ,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,1467811795,Mon Apr 06 22:20:05 PDT 2009,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it
9,0,1467812025,Mon Apr 06 22:20:09 PDT 2009,NO_QUERY,mimismo,@twittera que me muera ?


In [ ]:
data_set.describe()

,polarity,id
count,1600000.0,1.600000e+06
mean,0.5,1.998818e+09
std,0.5,1.935761e+08
min,0.0,1.467810e+09
25%,0.0,1.956916e+09
50%,0.5,2.002102e+09
75%,1.0,2.177059e+09
max,1.0,2.329206e+09


In [ ]:
# check the number of positive vs. negative tagged sentences
positives = data_set['polarity'][data_set.polarity == 1 ]
negatives = data_set['polarity'][data_set.polarity == 0 ]

print('Total length of the data is:         {}'.format(data_set.shape[0]))
print('No. of positve tagged sentences is:  {}'.format(len(positives)))
print('No. of negative tagged sentences is: {}'.format(len(negatives)))

Total length of the data is:         1600000
No. of positve tagged sentences is:  800000
No. of negative tagged sentences is: 800000


In [ ]:
# get a word count of text
def word_count(words):
    return len(words.split())

In [ ]:
# plot word count distribution for both positive and negative
'''
data_set['word count'] = data_set['text'].apply(word_count)
p = data_set['word count'][data_set.polarity == 1]
n =data_set['word count'][data_set.polarity == 0]
plt.figure(figsize=(12,6))
plt.xlim(0,45)
plt.xlabel('Word count')
plt.ylabel('Frequency')
g = plt.hist([p, n], color=['g','r'], alpha=0.5, label=['positive','negative'])
plt.legend(loc='upper right')
'''

"\ndata_set['word count'] = data_set['text'].apply(word_count)\np = data_set['word count'][data_set.polarity == 1]\nn =data_set['word count'][data_set.polarity == 0]\nplt.figure(figsize=(12,6))\nplt.xlim(0,45)\nplt.xlabel('Word count')\nplt.ylabel('Frequency')\ng = plt.hist([p, n], color=['g','r'], alpha=0.5, label=['positive','negative'])\nplt.legend(loc='upper right')\n"

In [ ]:
# get common words in training dataset
from collections import Counter
all_words = []
for line in list(data_set['text']):
    words = line.split()
    for word in words:
      if(len(word)>2):
        all_words.append(word.lower())


Counter(all_words).most_common(20)

[('the', 518734),
 ('and', 295675),
 ('you', 237766),
 ('for', 214051),
 ('have', 143295),
 ('that', 128775),
 ("i'm", 127616),
 ('but', 124676),
 ('just', 124454),
 ('with', 113488),
 ('was', 102742),
 ('not', 102160),
 ('this', 88151),
 ('get', 80943),
 ('good', 77367),
 ('are', 75994),
 ('like', 75424),
 ('all', 74076),
 ('out', 68953),
 ('your', 64854)]

In [ ]:
''''
%matplotlib inline
sns.countplot(data_set['polarity'])
'''

"'\n%matplotlib inline\nsns.countplot(data_set['polarity'])\n"

In [ ]:
# Dropping the columns which are not required
data_set.drop(['date','query','user'], axis=1, inplace=True)
data_set.drop('id', axis=1, inplace=True)
data_set.head(10)

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew
6,0,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,@Tatiana_K nope they didn't have it
9,0,@twittera que me muera ?


In [ ]:
# Checking for null values
data_set.isnull().sum()


polarity    0
text        0
dtype: int64

In [ ]:
# Typecasting pandas object to a string type
data_set['text'] = data_set['text'].astype('str')


In [ ]:
stopword = set(stopwords.words('english'))
print(stopword)

{'have', "aren't", 'after', 'having', 'in', 'when', 'it', 'has', 'had', "won't", "wouldn't", "don't", 'the', 'of', 'between', 'each', "doesn't", 'ma', 'down', "you'll", 'we', 'yourself', 'further', 'should', 'was', 'own', 'such', 'or', 're', 'needn', "shan't", "hasn't", 'up', "wasn't", "hadn't", 'mustn', 'hasn', "weren't", 'off', 'won', 'theirs', 'ours', 'both', 'himself', 'haven', 'herself', 'are', 'o', "haven't", 'against', 'not', 'this', 'her', 'd', 'why', 'only', 'very', 'did', 've', 'by', "mustn't", 'how', 'shouldn', 't', 'our', 'while', "she's", 'their', 'm', 'ourselves', 'who', "that'll", 'he', 'do', 'ain', "it's", "needn't", 'shan', "isn't", 'out', 'some', 'his', 'will', 'just', 'myself', 'nor', 'hadn', 'my', 'him', 'which', 'there', 'with', 'can', 'itself', 'now', 'you', 'been', 'an', 'these', 'for', 'to', 'during', 'than', 'as', 'i', 'under', 'more', 'weren', 'over', 'few', 'wouldn', 'any', 'again', 'a', 'whom', 'then', 'those', 'yours', 'didn', 'hers', 'all', 'if', 'here', '

# The Preprocessing steps taken are:

Lower Casing: Each text is converted to lowercase.
Removing URLs: Links starting with "http" or "https" or "www" are replaced by "".

Removing Usernames: Replace @Usernames with word "". (eg: "@XYZ" to "")

Removing Short Words: Words with length less than 2 are removed.

Removing Stopwords: Stopwords are the English words which does not add much meaning to a sentence. They can safely be ignored without sacrificing the meaning of the sentence. (eg: "the", "he", "have")

Lemmatizing: Lemmatization is the process of converting a word to its base form. (e.g: “wolves” to “wolf”)

In [ ]:


urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = '@[^\s]+'
def process_tweets(tweet):
  # Lower Casing
    tweet = tweet.lower()
    tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
        if len(w)>1:
            word = wordLemm.lemmatize(w)
            finalwords.append(word)
    return ' '.join(finalwords)

In [ ]:
data_set['processed_tweets'] = data_set['text'].apply(lambda x: process_tweets(x))
print('Text Preprocessing complete.')

Text Preprocessing complete.


In [ ]:
data_set.head()

,polarity,text,processed_tweets
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot awww thats bummer shoulda got david...
1,0,is upset that he can't update his Facebook by ...,upset cant update facebook texting might cry r...
2,0,@Kenichan I dived many times for the ball. Man...,kenichan dived many time ball managed save 50 ...
3,0,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving im mad cant see


# Analizing the data

# # Positive word cloud

In [ ]:
'''
plt.figure(figsize = (15,15))
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(data_set[data_set.polarity == 1].processed_tweets))
plt.imshow(wc , interpolation = 'bilinear')

'''

'\nplt.figure(figsize = (15,15)) \nwc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(data_set[data_set.polarity == 1].processed_tweets))\nplt.imshow(wc , interpolation = \'bilinear\')\n\n'

# #Negetive word Cloud

In [ ]:
'''
plt.figure(figsize = (15,15))
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(data_set[data_set.polarity == 0].processed_tweets))
plt.imshow(wc , interpolation = 'bilinear')
'''

'\nplt.figure(figsize = (15,15)) \nwc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(data_set[data_set.polarity == 0].processed_tweets))\nplt.imshow(wc , interpolation = \'bilinear\')\n'

# Vectorization and Splitting the data
Storing input variable-processes_tweets to X and output variable-polarity to y

In [ ]:
X = data_set['processed_tweets'].values
y = data_set['polarity'].values

In [ ]:
print(X.shape)
print(y.shape)

(1600000,)
(1600000,)


# Convert text to word frequency vectors
TF-IDF
This is an acronym than stands for Term Frequency – Inverse Document Frequency which are the components of the resulting scores assigned to each word.

Term Frequency: This summarizes how often a given word appears within a document.
Inverse Document Frequency: This downscales words that appear a lot across documents.

In [ ]:
#Convert a collection of raw documents to a matrix of TF-IDF features.
vector = TfidfVectorizer(sublinear_tf=True)
X = vector.fit_transform(X)
print(f'Vector fitted.')
print('No. of feature_words: ', len(vector.get_feature_names()))

Vector fitted.
No. of feature_words:  762358


In [ ]:
print(X.shape)
print(y.shape)

(1600000, 762358)
(1600000,)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=101)

In [ ]:
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print()
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (1280000, 762358)
y_train (1280000,)

X_test (320000, 762358)
y_test (320000,)


# Model Building

Model evaluating function

In [ ]:
def model_Evaluate(model):
    #accuracy of model on training data
    acc_train=model.score(X_train, y_train)
    #accuracy of model on test data
    acc_test=model.score(X_test, y_test)

    print('Accuracy of model on training set : {}'.format(acc_train*100))
    print('Accuracy of model on test set : {} \n'.format(acc_test*100))

    # Predict values for Test dataset
    y_pred = model.predict(X_test)

    # Print the evaluation metrics for the dataset.
    print(classification_report(y_test, y_pred))

# Logistic Regression

In [ ]:
lg = LogisticRegression()
history=lg.fit(X_train, y_train)
model_Evaluate(lg)

Accuracy of model on training set : 82.29023437500001
Accuracy of model on test set : 78.50281249999999 

              precision    recall  f1-score   support

           0       0.80      0.76      0.78    159763
           1       0.77      0.81      0.79    160237

    accuracy                           0.79    320000
   macro avg       0.79      0.78      0.78    320000
weighted avg       0.79      0.79      0.78    320000



# Linear SVM

In [ ]:
svm = LinearSVC()
svm.fit(X_train, y_train)
model_Evaluate(svm)

Accuracy of model on training set : 89.192578125
Accuracy of model on test set : 77.84687500000001 

              precision    recall  f1-score   support

           0       0.79      0.76      0.77    159763
           1       0.77      0.79      0.78    160237

    accuracy                           0.78    320000
   macro avg       0.78      0.78      0.78    320000
weighted avg       0.78      0.78      0.78    320000



# Random Forest

In [ ]:
rf = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', max_depth=50)
rf.fit(X_train, y_train)
model_Evaluate(rf)

Accuracy of model on training set : 70.826953125
Accuracy of model on test set : 69.3453125 

              precision    recall  f1-score   support

           0       0.73      0.61      0.67    159763
           1       0.67      0.78      0.72    160237

    accuracy                           0.69    320000
   macro avg       0.70      0.69      0.69    320000
weighted avg       0.70      0.69      0.69    320000



# Naive Bayes

In [ ]:
nb = BernoulliNB()
nb.fit(X_train, y_train)
model_Evaluate(nb)

Accuracy of model on training set : 83.894296875
Accuracy of model on test set : 77.2934375 

              precision    recall  f1-score   support

           0       0.77      0.79      0.78    159763
           1       0.78      0.76      0.77    160237

    accuracy                           0.77    320000
   macro avg       0.77      0.77      0.77    320000
weighted avg       0.77      0.77      0.77    320000



# LSTM

> Indented block



In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
import re

In [ ]:
import keras
keras.__version__

'2.4.3'

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 5000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_set.processed_tweets)
sequences = tokenizer.texts_to_sequences(data_set.processed_tweets)
tweets = pad_sequences(sequences, maxlen=max_len)
print(tweets)

[[   0    0    0 ...  759 1865    2]
 [   0    0    0 ...   11  187 1283]
 [   0    0    0 ...  368    6 3132]
 ...
 [   0    0    0 ...  123  504 1746]
 [   0    0    0 ...  415 4997   12]
 [   0    0    0 ...    0    0  458]]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweets, data_set.polarity.values, test_size=0.2, random_state=101)

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
model2 = Sequential()
model2.add(layers.Embedding(max_words, 128))
model2.add(layers.LSTM(64,dropout=0.5))
model2.add(layers.Dense(16, activation='relu'))
model2.add(layers.Dense(8, activation='relu'))
model2.add(layers.Dense(1,activation='sigmoid'))
model2.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
checkpoint2 = ModelCheckpoint("rnn_model.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=10,validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/10
40000/40000 [==============================] - 4535s 113ms/step - loss: 0.4900 - accuracy: 0.7606 - val_loss: 0.4560 - val_accuracy: 0.7826

Epoch 00001: val_accuracy improved from -inf to 0.78262, saving model to rnn_model.hdf5
Epoch 2/10
 6874/40000 [====>.........................] - ETA: 58:12 - loss: 0.4516 - accuracy: 0.7871